<hr>
<center><h1>Schnorr Signature over Elliptic Curve</h1></center>
<hr>

In [34]:
import hashlib

## Curve P-256

In [35]:
# a large prime number p
p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff
# the finite field of Z/pZ
K = GF(p)
# equation parameters a and b such as : y^2 = x^3 + ax + b
a = K(0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc)
b = K(0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b)
# Define the EC 
E = EllipticCurve(K, (a, b))
# the generator of the EC
G = E(0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296, 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5)
n = 0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551
E.set_order(n)

## 1. KeyGen - Original Key Pair

In [42]:
# Alice chooses a random d : 1 < d < n
d = 0xffff  # fixed for demonstration
# public key P
P = d * G
P,d

((109588801698256994951990571109052358531364605021783910115998727948928103999695 : 82091379372138771455752671260931004134817620445688054187650988778738847793385 : 1),
 65535)

## 2. Sign - Generate Valid Signature

In [41]:
# choose k : 1 < k < n
k = int(K.random_element(1, n))
# calculate R = k*G
R = k * G
r = int(R[0])  # x-coordinate of R

# a given message m 
m = "Schnorr Signature"

# compute e = hash(r || m) and convert to integer
concat_str = str(r) + m
hash_bytes = hashlib.sha256(concat_str.encode('utf-8')).digest()
e = int.from_bytes(hash_bytes, 'big') % n

# compute s = (k - e * d) mod n
s = (k - e * d) % n

signature = (r, s)
signature, e

((46563057759920574265813633937021237484209421603266754390983743901188948891124,
  79300363929158429744481005666745093268220220092726879819017051691755264814493),
 95589711530293888627388679274723937796066900166694083381185511481685920805490)

## 3. Verify - Original Signature

In [38]:
r, s = signature

# compute e = hash(r || m)
concat_str = str(r) + m
hash_bytes = hashlib.sha256(concat_str.encode('utf-8')).digest()
e = int.from_bytes(hash_bytes, 'big') % n

# compute Rv = s*G + e*P
Rv = s * G + e * P

# verify: r == x-coordinate of Rv mod n
valid = (int(Rv[0]) % n) == r
print(f"Original signature valid: {valid}")

Original signature valid: True


## 4. Attack: Signature Malleability

**Démonstration:** Supposons (r,s) une signature valide. Un attaquant peut construire une signature valide (r, s+a*e) pour la clé publique P - a*G

In [48]:
# Original signature and public key
r, s = signature
print(f"Original signature: (r={r}, s={s})")
print(f"Original public key P: {P}\n")

print('-'*100)
# Choose  a
a = 12345  # random value chosen by attacker
print(f"Attacker chooses a = {a}")

# Compute e = hash(r||m)
concat_str = str(r) + m
hash_bytes = hashlib.sha256(concat_str.encode('utf-8')).digest()
e = int.from_bytes(hash_bytes, 'big') % n

# create new signature and public key
s_prime = (s + a * e) % n  # modified signature
P_prime = P - a * G        # modified public key

print(f"\nAttacker creates:")
print(f"  Modified signature: (r={r}, s'={s_prime})")
print(f"  Modified public key: P' = P - {a}*G = {P_prime}")

# Verify the modified signature 
Rv_prime = s_prime * G + e * P_prime
valid_attack = (int(Rv_prime[0]) % n) == r

print('-'*100)
print(f"\nVerifying with (r, s', P'):")
print(f"  Rv' = s'*G + e*P' = {Rv_prime}")
print(f"  x-coordinate: {int(Rv_prime[0]) % n}")
print(f"  Original r: {r}")
print(f"  Signature valide: {valid_attack}")

# Verify original signature still works with original key
Rv_orig = s * G + e * P
valid_orig = (int(Rv_orig[0]) % n) == r

print('-'*100)
print(f"\nVerification originale sig (r, s, P): {valid_orig}")
print(f"Verification modified sig (r, s', P'): {valid_attack}")

Original signature: (r=46563057759920574265813633937021237484209421603266754390983743901188948891124, s=79300363929158429744481005666745093268220220092726879819017051691755264814493)
Original public key P: (109588801698256994951990571109052358531364605021783910115998727948928103999695 : 82091379372138771455752671260931004134817620445688054187650988778738847793385 : 1)

----------------------------------------------------------------------------------------------------
Attacker chooses a = 12345

Attacker creates:
  Modified signature: (r=46563057759920574265813633937021237484209421603266754390983743901188948891124, s'=97108062666682394208044790721175341515132088763652570928914201755241364424064)
  Modified public key: P' = P - 12345*G = (55515772619765937206544089306311119981104869482570306062592471106548284121367 : 102782000583052235095954056552530504010483885716354654645368375319099404273982 : 1)
----------------------------------------------------------------------------------------